#### Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

##### Data Directory

In [2]:
from pathlib import Path


DATA_PATH = Path('/home/sayem/Desktop/Project/data')
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')


##### S&P 500 Prices

In [3]:
df = web.DataReader(name='SP500', data_source='fred', start=2009).squeeze().to_frame('close')
print(df.info())
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/fred', df)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2610 entries, 2013-09-09 to 2023-09-08
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   2518 non-null   float64
dtypes: float64(1)
memory usage: 40.8 KB
None


In [4]:
sp500_stooq = (pd.read_csv(DATA_PATH / '^spx_d.csv', index_col=0,
                     parse_dates=True).loc['1950':'2019'].rename(columns=str.lower))
                     
print(sp500_stooq.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17700 entries, 1950-01-03 to 2019-12-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    17700 non-null  float64
 1   high    17700 non-null  float64
 2   low     17700 non-null  float64
 3   close   17700 non-null  float64
 4   volume  17700 non-null  float64
dtypes: float64(5)
memory usage: 829.7 KB
None


##### S&P 500 Constituents

In [5]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]

# Drop rows containing NaN values
df_cleaned = df.dropna()

In [6]:
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [7]:
df.columns = ['ticker', 'name', 'gics_sector', 'gics_sub_industry',
              'location', 'first_added', 'cik', 'founded']
df = df.set_index('ticker')

In [8]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stocks', df)

##### Metadata on US-traded companies

In [9]:
from pathlib import Path
import pandas as pd

csv_folder = DATA_PATH / 'csv'

# List all files ending with _screener.csv in the csv folder
csv_files = list(csv_folder.glob('*_screener.csv'))

# Load all these CSVs into a DataFrame
all_data = [pd.read_csv(file) for file in csv_files]
df = pd.concat(all_data, ignore_index=True)

# Rename columns, set index, and remove duplicates
df = df.rename(columns=str.lower).set_index('symbol')
df = df[~df.index.duplicated()]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7516 entries, AACG to ZWS
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        7516 non-null   object 
 1   last sale   7516 non-null   object 
 2   net change  7516 non-null   float64
 3   % change    7512 non-null   object 
 4   market cap  7078 non-null   float64
 5   country     7092 non-null   object 
 6   ipo year    4339 non-null   float64
 7   volume      7516 non-null   int64  
 8   sector      6876 non-null   object 
 9   industry    6876 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 645.9+ KB


In [10]:
df

,name,last sale,net change,% change,market cap,country,ipo year,volume,sector,industry
symbol,,,,,,,,,,
AACG,ATA Creativity Global American Depositary Shares,$1.2901,0.0401,3.208%,4.079576e+07,China,2008.0,9180,Consumer Discretionary,Educational Services
AACI,Armada Acquisition Corp. I Common Stock,$10.53,-0.0500,-0.473%,0.000000e+00,United States,2021.0,3415,Finance,Blank Checks
AACIW,Armada Acquisition Corp. I Warrant,$0.0999,-0.0001,-0.10%,0.000000e+00,United States,2021.0,2400,Finance,Blank Checks
AADI,Aadi Bioscience Inc. Common Stock,$5.91,0.1300,2.249%,1.449124e+08,United States,NaN,115082,Health Care,Biotechnology: Pharmaceutical Preparations
AAL,American Airlines Group Inc. Common Stock,$15.65,-0.1100,-0.698%,1.022514e+10,United States,NaN,16049370,Consumer Discretionary,Air Freight/Delivery Services
...,...,...,...,...,...,...,...,...,...,...
ZTR,Virtus Total Return Fund Inc.,$5.99,0.0300,0.503%,0.000000e+00,United States,1988.0,182795,Finance,Investment Managers
ZTS,Zoetis Inc. Class A Common Stock,$190.83,1.7300,0.915%,8.818486e+10,United States,2013.0,1742920,Health Care,Biotechnology: Pharmaceutical Preparations
ZUO,Zuora Inc. Class A Common Stock,$10.45,0.0300,0.288%,1.433740e+09,United States,2018.0,413182,Technology,EDP Services


In [11]:
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
Index: 7516 entries, AACG to ZWS
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        7516 non-null   object 
 1   last sale   7516 non-null   object 
 2   net change  7516 non-null   float64
 3   % change    7512 non-null   object 
 4   market cap  7078 non-null   float64
 5   country     7092 non-null   object 
 6   ipo year    4339 non-null   float64
 7   volume      7516 non-null   int64  
 8   sector      6876 non-null   object 
 9   industry    6876 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 645.9+ KB
None
